In [24]:
import numpy as np
from sklearn import preprocessing
from numpy import random
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
import pandas as pd
import datetime
# import skimage.io as io
# import skimage
import math
from sklearn.decomposition import PCA
import scipy.io as sio

def show_accuracy(predictLabel,Label):
    Label = np.ravel(Label).tolist()
    predictLabel = predictLabel.tolist()
    count = 0
    for i in range(len(Label)):
        if Label[i] == predictLabel[i]:
            count += 1
    return (round(count/len(Label),5))

class node_generator:
    def __init__(self,whiten = False):
        self.Wlist = []
        self.blist = []
        self.nonlinear = 0
        self.whiten = whiten
    
    def sigmoid(self,data):
        return 1.0/(1+np.exp(-data))
    
    def linear(self,data):
        return data
    
    def tanh(self,data):
        return (np.exp(data)-np.exp(-data))/(np.exp(data)+np.exp(-data))
    
    def relu(self,data):
        return np.maximum(data,0)
    
    def orth(self,W):
        for i in range(0,W.shape[1]):
            w = np.mat(W[:,i].copy()).T
            w_sum = 0
            for j in range(i):
                wj = np.mat(W[:,j].copy()).T
                w_sum += (w.T.dot(wj))[0,0]*wj 
            w -= w_sum
            w = w/np.sqrt(w.T.dot(w))
            W[:,i] = np.ravel(w)
        return W
        
    def generator(self,shape,times):
        for i in range(times):
            W = 2*random.random(size=shape)-1
            if self.whiten == True:
                W = self.orth(W)
            b = 2*random.random()-1
            yield (W,b)
    
    def generator_nodes(self, data, times, batchsize, nonlinear):
        self.Wlist = [elem[0] for elem in self.generator((data.shape[1],batchsize),times)]
        self.blist = [elem[1] for elem in self.generator((data.shape[1],batchsize),times)]
        
        self.nonlinear = {'linear':self.linear,
                          'sigmoid':self.sigmoid,
                          'tanh':self.tanh,
                          'relu':self.relu
                          }[nonlinear]
        nodes = self.nonlinear(data.dot(self.Wlist[0])+self.blist[0])
        for i in range(1,len(self.Wlist)):
            nodes = np.column_stack((nodes, self.nonlinear(data.dot(self.Wlist[i])+self.blist[i])))
        return nodes
        
    def transform(self,testdata):
        testnodes = self.nonlinear(testdata.dot(self.Wlist[0])+self.blist[0])
        for i in range(1,len(self.Wlist)):
            testnodes = np.column_stack((testnodes, self.nonlinear(testdata.dot(self.Wlist[i])+self.blist[i])))
        return testnodes   

    def update(self,otherW, otherb):
        self.Wlist += otherW
        self.blist += otherb
        
class scaler:
    def __init__(self):
        self._mean = 0
        self._std = 0
    
    def fit_transform(self,traindata):
        self._mean = traindata.mean(axis = 0)
        self._std = traindata.std(axis = 0)
        return (traindata-self._mean)/(self._std+0.001)
    
    def transform(self,testdata):
        return (testdata-self._mean)/(self._std+0.001)
        

class broadnet:
    def __init__(self, 
                 maptimes = 10, 
                 enhencetimes = 10,
                 map_function = 'linear',
                 enhence_function = 'linear',
                 batchsize = 'auto', 
                 reg = 0.001):
        
        self._maptimes = maptimes
        self._enhencetimes = enhencetimes
        self._batchsize = batchsize
        self._reg = reg
        self._map_function = map_function
        self._enhence_function = enhence_function
        
        self.W = 0
        self.pesuedoinverse = 0
        self.normalscaler = scaler()
        self.onehotencoder = preprocessing.OneHotEncoder(sparse = False)
        self.mapping_generator = node_generator()
        self.enhence_generator = node_generator(whiten = True)

    def fit(self,data,label,c):
        if self._batchsize == 'auto':
            self._batchsize = data.shape[1]
        data = self.normalscaler.fit_transform(data)
        label1=label
        label = self.onehotencoder.fit_transform(np.mat(label).T)
        
        mappingdata = self.mapping_generator.generator_nodes(data,self._maptimes,self._batchsize,self._map_function)
        enhencedata = self.enhence_generator.generator_nodes(mappingdata,self._enhencetimes,self._batchsize,self._enhence_function)
        
        print('number of mapping nodes {0}, number of enhence nodes {1}'.format(mappingdata.shape[1],enhencedata.shape[1]))
        print('mapping nodes maxvalue {0} minvalue {1} '.format(round(np.max(mappingdata),5),round(np.min(mappingdata),5)))
        print('enhence nodes maxvalue {0} minvalue {1} '.format(round(np.max(enhencedata),5),round(np.min(enhencedata),5)))
        
        inputdata = np.column_stack((mappingdata,enhencedata))
        S = self.LDA_dimensionality(inputdata,label1)
        pesuedoinverse = self.pinv(inputdata,self._reg,S,c)
        self.W =  pesuedoinverse.dot(label)
            
    def pinv(self,A,reg,S,c):
        return np.mat(reg*np.eye(A.shape[1])+A.T.dot(A)+c*S).I.dot(A.T)
    
    def LDA_dimensionality(self,X, y):
        '''
        X为数据集，y为label，k为目标维数
        '''
        label_ = list(set(y))

        X_classify = {}

        for label in label_:
            X1 = np.array([X[i] for i in range(len(X)) if y[i] == label])
            X_classify[label] = X1

            mju = np.mean(X, axis=0)
            mju_classify = {}

        for label in label_:
            mju1 = np.mean(X_classify[label], axis=0)
            mju_classify[label] = mju1

            #St = np.dot((X - mju).T, X - mju)

            Sw = np.zeros((len(mju), len(mju)))  # 计算类内散度矩阵
        
        for i in label_:
            Sw += np.dot((X_classify[i] - mju_classify[i]).T,
                         X_classify[i] - mju_classify[i])

        # Sb=St-Sw

        Sb = np.zeros((len(mju), len(mju)))  # 计算类间散度矩阵
        for i in label_:
            Sb += len(X_classify[i]) * np.dot((mju_classify[i] - mju).reshape(
                (len(mju), 1)), (mju_classify[i] - mju).reshape((1, len(mju))))

        return (Sw-Sb)

        
    
    def decode(self,Y_onehot):
        Y = []
        for i in range(Y_onehot.shape[0]):
            lis = np.ravel(Y_onehot[i,:]).tolist()
            Y.append(lis.index(max(lis)))
        return np.array(Y)
    
    def accuracy(self,predictlabel,label):
        label = np.ravel(label).tolist()
        predictlabel = predictlabel.tolist()
        count = 0
        for i in range(len(label)):
            if label[i] == predictlabel[i]:
                count += 1
        return (round(count/len(label),5))
        
    def predict(self,testdata):
        testdata = self.normalscaler.transform(testdata)
        test_mappingdata = self.mapping_generator.transform(testdata)
        test_enhencedata = self.enhence_generator.transform(test_mappingdata)
        
        test_inputdata = np.column_stack((test_mappingdata,test_enhencedata))    
        return self.decode(test_inputdata.dot(self.W))    

In [25]:
import scipy.io as scio 
import scipy.io as sio
from sklearn.metrics import confusion_matrix,accuracy_score,cohen_kappa_score
from operator import truediv 

def AA_andEachClassAccuracy(confusion_matrix):
    counter = confusion_matrix.shape[0]
    list_diag = np.diag(confusion_matrix)
    list_raw_sum = np.sum(confusion_matrix, axis=1)
    each_acc = np.nan_to_num(truediv(list_diag, list_raw_sum))
    average_acc = np.mean(each_acc)
    return each_acc, average_acc

def reports (y_pred,y_test):
    oa = accuracy_score(y_test, y_pred)
    confusion = confusion_matrix(y_test, y_pred)
    each_acc, aa = AA_andEachClassAccuracy(confusion)
    kappa = cohen_kappa_score(y_test, y_pred)
    
    return oa, each_acc, aa, kappa


TE= scio.loadmat('C:/Users/yonghe/Desktop/IEEE_TGRS_GCN-master/IEEE_TGRS_GCN-master/TE_map.mat')
trainlabel = TE['TrLabel']
testlabel = TE['TeLabel']
Test_map=TE['imagemap']
test_SL=TE['test_SL']

TrLabel = TE['TrLabel']
TeLabel = TE['TeLabel']
testdata=scio.loadmat ('C:/Users/yonghe/Desktop/IEEE_TGRS_GCN-master/IEEE_TGRS_GCN-master/testfeature.mat')
traindata=scio.loadmat('C:/Users/yonghe/Desktop/IEEE_TGRS_GCN-master/IEEE_TGRS_GCN-master/trainfeature.mat')
testdata = testdata['testfeature']
traindata = traindata['trainfeature']
testlabel=np.transpose(testlabel)
trainlabel=np.transpose(trainlabel)
testlabel=testlabel[0]
trainlabel=trainlabel[0]
e=0



for i in np.arange(-10,11,1):
    for c in np.arange(-10,11,1):
        bls = broadnet(maptimes = 10, 
                   enhencetimes = 10,
                   map_function = 'relu',
                   enhence_function = 'relu',
                   batchsize = 100, 
                   reg = np.float_power(10,i))

        starttime = datetime.datetime.now()
        bls.fit(traindata,trainlabel,np.float_power(10,c))
        endtime = datetime.datetime.now()
        print('the training time of BLS is {0} seconds'.format((endtime - starttime).total_seconds()))

        predictlabel = bls.predict(testdata)
        GCNBLSpredictlabel=predictlabel+1
        oa, each_acc, aa, kappa=reports (GCNBLSpredictlabel,testlabel)
        if oa>e:
            e=oa
            GCNBLSpredictlabel1=GCNBLSpredictlabel
oa, each_acc, aa, kappa=reports (GCNBLSpredictlabel1,testlabel)
num_classes = np.max(testlabel)
GCNBLSoutputs = np.zeros([num_classes+3])
GCNBLSoutputs[0:num_classes] = each_acc
GCNBLSoutputs[num_classes] = oa
GCNBLSoutputs[num_classes+1] = aa
GCNBLSoutputs[num_classes+2] = kappa
a,n=Test_map.shape
Test_map=Test_map.reshape(1,a*n)
Test_map[0,test_SL[0,:]]=test_SL[1,:]
sio.savemat('GCNBLSoutputs.mat', {'GCNBLSoutputs': GCNBLSoutputs,'Test_map': Test_map})
print(GCNBLSoutputs[num_classes])

number of mapping nodes 1000, number of enhence nodes 1000
mapping nodes maxvalue 47.59265 minvalue 0.0 
enhence nodes maxvalue 25.29078 minvalue 0.0 
the training time of BLS is 2.868542 seconds
number of mapping nodes 1000, number of enhence nodes 1000
mapping nodes maxvalue 41.80712 minvalue 0.0 
enhence nodes maxvalue 30.2004 minvalue 0.0 
the training time of BLS is 2.024215 seconds
number of mapping nodes 1000, number of enhence nodes 1000
mapping nodes maxvalue 40.89348 minvalue 0.0 
enhence nodes maxvalue 23.67587 minvalue 0.0 
the training time of BLS is 2.040707 seconds
number of mapping nodes 1000, number of enhence nodes 1000
mapping nodes maxvalue 41.02626 minvalue 0.0 
enhence nodes maxvalue 28.66995 minvalue 0.0 
the training time of BLS is 2.030063 seconds
number of mapping nodes 1000, number of enhence nodes 1000
mapping nodes maxvalue 38.65122 minvalue 0.0 
enhence nodes maxvalue 32.92661 minvalue 0.0 
the training time of BLS is 2.029693 seconds
number of mapping nod

number of mapping nodes 1000, number of enhence nodes 1000
mapping nodes maxvalue 41.73592 minvalue 0.0 
enhence nodes maxvalue 26.99298 minvalue 0.0 
the training time of BLS is 2.015088 seconds
number of mapping nodes 1000, number of enhence nodes 1000
mapping nodes maxvalue 38.53704 minvalue 0.0 
enhence nodes maxvalue 29.45867 minvalue 0.0 
the training time of BLS is 2.049808 seconds
number of mapping nodes 1000, number of enhence nodes 1000
mapping nodes maxvalue 37.04686 minvalue 0.0 
enhence nodes maxvalue 36.37761 minvalue 0.0 
the training time of BLS is 2.009304 seconds
number of mapping nodes 1000, number of enhence nodes 1000
mapping nodes maxvalue 50.66546 minvalue 0.0 
enhence nodes maxvalue 25.47064 minvalue 0.0 
the training time of BLS is 2.034472 seconds
number of mapping nodes 1000, number of enhence nodes 1000
mapping nodes maxvalue 42.81993 minvalue 0.0 
enhence nodes maxvalue 29.01217 minvalue 0.0 
the training time of BLS is 2.028139 seconds
number of mapping no

number of mapping nodes 1000, number of enhence nodes 1000
mapping nodes maxvalue 35.91938 minvalue 0.0 
enhence nodes maxvalue 30.48808 minvalue 0.0 
the training time of BLS is 2.025731 seconds
number of mapping nodes 1000, number of enhence nodes 1000
mapping nodes maxvalue 36.29277 minvalue 0.0 
enhence nodes maxvalue 23.62946 minvalue 0.0 
the training time of BLS is 2.039948 seconds
number of mapping nodes 1000, number of enhence nodes 1000
mapping nodes maxvalue 45.72734 minvalue 0.0 
enhence nodes maxvalue 29.05053 minvalue 0.0 
the training time of BLS is 2.055461 seconds
number of mapping nodes 1000, number of enhence nodes 1000
mapping nodes maxvalue 40.11308 minvalue 0.0 
enhence nodes maxvalue 28.55832 minvalue 0.0 
the training time of BLS is 2.050513 seconds
number of mapping nodes 1000, number of enhence nodes 1000
mapping nodes maxvalue 48.82229 minvalue 0.0 
enhence nodes maxvalue 21.83333 minvalue 0.0 
the training time of BLS is 1.997521 seconds
number of mapping no

number of mapping nodes 1000, number of enhence nodes 1000
mapping nodes maxvalue 39.89139 minvalue 0.0 
enhence nodes maxvalue 28.90594 minvalue 0.0 
the training time of BLS is 2.038343 seconds
number of mapping nodes 1000, number of enhence nodes 1000
mapping nodes maxvalue 38.79086 minvalue 0.0 
enhence nodes maxvalue 24.89892 minvalue 0.0 
the training time of BLS is 2.123646 seconds
number of mapping nodes 1000, number of enhence nodes 1000
mapping nodes maxvalue 43.44862 minvalue 0.0 
enhence nodes maxvalue 29.49819 minvalue 0.0 
the training time of BLS is 1.993807 seconds
number of mapping nodes 1000, number of enhence nodes 1000
mapping nodes maxvalue 39.12568 minvalue 0.0 
enhence nodes maxvalue 30.94451 minvalue 0.0 
the training time of BLS is 2.002507 seconds
number of mapping nodes 1000, number of enhence nodes 1000
mapping nodes maxvalue 39.50437 minvalue 0.0 
enhence nodes maxvalue 27.39941 minvalue 0.0 
the training time of BLS is 1.990939 seconds
number of mapping no

number of mapping nodes 1000, number of enhence nodes 1000
mapping nodes maxvalue 42.1131 minvalue 0.0 
enhence nodes maxvalue 29.42332 minvalue 0.0 
the training time of BLS is 1.984043 seconds
number of mapping nodes 1000, number of enhence nodes 1000
mapping nodes maxvalue 53.16014 minvalue 0.0 
enhence nodes maxvalue 28.40545 minvalue 0.0 
the training time of BLS is 2.019294 seconds
number of mapping nodes 1000, number of enhence nodes 1000
mapping nodes maxvalue 39.78559 minvalue 0.0 
enhence nodes maxvalue 36.79806 minvalue 0.0 
the training time of BLS is 1.960997 seconds
number of mapping nodes 1000, number of enhence nodes 1000
mapping nodes maxvalue 46.05681 minvalue 0.0 
enhence nodes maxvalue 30.10204 minvalue 0.0 
the training time of BLS is 1.973236 seconds
number of mapping nodes 1000, number of enhence nodes 1000
mapping nodes maxvalue 39.16603 minvalue 0.0 
enhence nodes maxvalue 28.9834 minvalue 0.0 
the training time of BLS is 1.959349 seconds
number of mapping node

...

number of mapping nodes 1000, number of enhence nodes 1000
mapping nodes maxvalue 39.67723 minvalue 0.0 
enhence nodes maxvalue 36.72262 minvalue 0.0 
the training time of BLS is 2.077318 seconds
number of mapping nodes 1000, number of enhence nodes 1000
mapping nodes maxvalue 39.04465 minvalue 0.0 
enhence nodes maxvalue 29.22157 minvalue 0.0 
the training time of BLS is 2.040926 seconds
number of mapping nodes 1000, number of enhence nodes 1000
mapping nodes maxvalue 38.53672 minvalue 0.0 
enhence nodes maxvalue 29.65241 minvalue 0.0 
the training time of BLS is 2.069197 seconds
number of mapping nodes 1000, number of enhence nodes 1000
mapping nodes maxvalue 40.59286 minvalue 0.0 
enhence nodes maxvalue 27.38327 minvalue 0.0 
the training time of BLS is 2.090285 seconds
number of mapping nodes 1000, number of enhence nodes 1000
mapping nodes maxvalue 39.90394 minvalue 0.0 
enhence nodes maxvalue 28.00218 minvalue 0.0 
the training time of BLS is 2.03212 seconds
number of mapping nod

number of mapping nodes 1000, number of enhence nodes 1000
mapping nodes maxvalue 40.24203 minvalue 0.0 
enhence nodes maxvalue 32.08897 minvalue 0.0 
the training time of BLS is 2.010527 seconds
number of mapping nodes 1000, number of enhence nodes 1000
mapping nodes maxvalue 41.31348 minvalue 0.0 
enhence nodes maxvalue 22.56633 minvalue 0.0 
the training time of BLS is 1.983939 seconds
number of mapping nodes 1000, number of enhence nodes 1000
mapping nodes maxvalue 41.21916 minvalue 0.0 
enhence nodes maxvalue 28.02052 minvalue 0.0 
the training time of BLS is 2.012111 seconds
number of mapping nodes 1000, number of enhence nodes 1000
mapping nodes maxvalue 39.50227 minvalue 0.0 
enhence nodes maxvalue 24.7345 minvalue 0.0 
the training time of BLS is 1.990222 seconds
number of mapping nodes 1000, number of enhence nodes 1000
mapping nodes maxvalue 38.48628 minvalue 0.0 
enhence nodes maxvalue 30.49115 minvalue 0.0 
the training time of BLS is 1.995412 seconds
number of mapping nod

number of mapping nodes 1000, number of enhence nodes 1000
mapping nodes maxvalue 35.17111 minvalue 0.0 
enhence nodes maxvalue 29.98076 minvalue 0.0 
the training time of BLS is 2.001363 seconds
number of mapping nodes 1000, number of enhence nodes 1000
mapping nodes maxvalue 40.21532 minvalue 0.0 
enhence nodes maxvalue 29.05991 minvalue 0.0 
the training time of BLS is 2.002658 seconds
number of mapping nodes 1000, number of enhence nodes 1000
mapping nodes maxvalue 36.3568 minvalue 0.0 
enhence nodes maxvalue 30.13458 minvalue 0.0 
the training time of BLS is 2.02296 seconds
number of mapping nodes 1000, number of enhence nodes 1000
mapping nodes maxvalue 40.05894 minvalue 0.0 
enhence nodes maxvalue 27.89083 minvalue 0.0 
the training time of BLS is 2.005091 seconds
number of mapping nodes 1000, number of enhence nodes 1000
mapping nodes maxvalue 43.57076 minvalue 0.0 
enhence nodes maxvalue 30.17512 minvalue 0.0 
the training time of BLS is 2.012749 seconds
number of mapping node

number of mapping nodes 1000, number of enhence nodes 1000
mapping nodes maxvalue 39.73214 minvalue 0.0 
enhence nodes maxvalue 26.12249 minvalue 0.0 
the training time of BLS is 1.99944 seconds
number of mapping nodes 1000, number of enhence nodes 1000
mapping nodes maxvalue 37.44694 minvalue 0.0 
enhence nodes maxvalue 26.59683 minvalue 0.0 
the training time of BLS is 1.976187 seconds
number of mapping nodes 1000, number of enhence nodes 1000
mapping nodes maxvalue 39.51536 minvalue 0.0 
enhence nodes maxvalue 25.48074 minvalue 0.0 
the training time of BLS is 1.991749 seconds
number of mapping nodes 1000, number of enhence nodes 1000
mapping nodes maxvalue 38.9592 minvalue 0.0 
enhence nodes maxvalue 24.74103 minvalue 0.0 
the training time of BLS is 1.997818 seconds
number of mapping nodes 1000, number of enhence nodes 1000
mapping nodes maxvalue 44.65185 minvalue 0.0 
enhence nodes maxvalue 33.34218 minvalue 0.0 
the training time of BLS is 2.045878 seconds
number of mapping node

number of mapping nodes 1000, number of enhence nodes 1000
mapping nodes maxvalue 40.11266 minvalue 0.0 
enhence nodes maxvalue 30.65978 minvalue 0.0 
the training time of BLS is 2.000699 seconds
number of mapping nodes 1000, number of enhence nodes 1000
mapping nodes maxvalue 34.69314 minvalue 0.0 
enhence nodes maxvalue 30.45172 minvalue 0.0 
the training time of BLS is 1.983399 seconds
number of mapping nodes 1000, number of enhence nodes 1000
mapping nodes maxvalue 57.47256 minvalue 0.0 
enhence nodes maxvalue 25.74126 minvalue 0.0 
the training time of BLS is 1.960656 seconds
number of mapping nodes 1000, number of enhence nodes 1000
mapping nodes maxvalue 38.7392 minvalue 0.0 
enhence nodes maxvalue 26.68924 minvalue 0.0 
the training time of BLS is 1.986359 seconds
number of mapping nodes 1000, number of enhence nodes 1000
mapping nodes maxvalue 39.75618 minvalue 0.0 
enhence nodes maxvalue 33.46235 minvalue 0.0 
the training time of BLS is 1.969905 seconds
number of mapping nod

In [27]:
sio.savemat('GCNDBLSoutputs.mat', {'GCNBLSoutputs': GCNBLSoutputs,'Test_map': Test_map})

In [26]:
GCNBLSoutputs

array([1.        , 0.99159664, 1.        , 0.91497976, 0.81410256,
       0.93303571, 0.94      , 1.        , 0.8368932 , 1.        ,
       1.        , 0.81325301, 1.        , 0.95141858, 0.94183545,
       0.94587653])

In [ ]:
for c in np.arange(-10,11,1):
    print(np.float_power(2,c))